**Multi Person Pose Estimation**

In [6]:
import os
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import mediapipe as mp
from mediapipe.python.solutions import pose as mp_pose

import torch

In [7]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

yolo_model.classes=[0]

C:\Users\Lenovo\anaconda3\lib\site-packages\torch\hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Lenovo/.cache\torch\hub\master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'setuptools>=70.0.0'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "gitpython>=3.1.30" "pillow>=10.3.0" "setuptools>=70.0.0" ' returned non-zero exit status 1.
     ------------------------------------ 207.3/207.3 kB 257.3 kB/s eta 0:00:00
     -------------------------------------- 62.7/62.7 kB 305.1 kB/s eta 0:00:00

requirements: AutoUpdate success  256.1s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'setuptools>=70.0.0']
requirements:  Restart runtime or rerun command for updates to take effect



YOLOv5  2024-7-20 Python-3.9.13 torch-2.2.1 CPU

100%|██████████████████████████████████████████████████████████████████████████████| 14.1M/14.1M [00:55<00:00, 267kB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_pose =mp.solutions.pose

In [15]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    h, w, _ = frame.shape
    size = (w, h)
    print(size)
    break

(640, 480)


In [17]:
cap = cv2.VideoCapture(0)

out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20, size)
while cap.isOpened():    
    ret, frame = cap.read()
    if not ret:
        break
        
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False    
    
    result = yolo_model(image)  
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    img_list =[]
    MARGIN=10

    for (xmin, ymin, xmax,   ymax,  confidence,  clas) in result.xyxy[0].tolist():
        with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
            results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:])

            mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                  ) 
            img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])

    out.write(image)
            
    cv2.imshow('Pose Estimation', image)
    if cv2.waitKey(10) & 0xFF == 27:
         break

cap.release()
out.release()
cv2.destroyAllWindows()